# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
import torch.nn.functional as F
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,5,5)#24
        self.conv2 = nn.Conv2d(5,10,3)#10
        self.fc1 = nn.Linear(10 * 10 * 10,200)
        self.fc2 = nn.Linear(200,10)
    def forward(self,x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out,2,2)#12
        out = self.conv2(out)
        out = F.relu(out)
        out = out.view(x.size(0),-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim = 1)
        return out
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNet().to(DEVICE)
from torchsummary import summary
summary(model,(1,28,28))#print layer shape
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 5, 24, 24]             130
            Conv2d-2           [-1, 10, 10, 10]             460
            Linear-3                  [-1, 200]         200,200
            Linear-4                   [-1, 10]           2,010
Total params: 202,800
Trainable params: 202,800
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.77
Estimated Total Size (MB): 0.81
----------------------------------------------------------------


Next, we can start to train and evaluate!

In [6]:
# train and evaluate
max_test_correct=0
corresponding_train_correct=0
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images,labels = images.to(DEVICE),labels.to(DEVICE)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    model.eval()
    train_loss = 0
    train_correct = 0
    with torch.no_grad():
        for images,labels in train_loader:
            images,labels = images.to(DEVICE),labels.to(DEVICE)
            output = model(images)
            train_loss += criterion(output,labels)
            pred = output.max(1,keepdim=True)[1]
            train_correct += pred.eq(labels.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    print('\ntrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    model.eval()
    test_loss = 0
    test_correct = 0
    with torch.no_grad():
        for images,labels in test_loader:
            images,labels = images.to(DEVICE),labels.to(DEVICE)
            output = model(images)
            test_loss += criterion(output,labels)
            pred = output.max(1,keepdim=True)[1]
            test_correct += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    if test_correct>max_test_correct:
        max_test_correct=test_correct
        corresponding_train_correct=train_correct
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))
    
    
    


100%|██████████| 468/468 [00:13<00:00, 35.27it/s]



train set: Average loss: 0.0001, Accuracy: 59722/60000 (99.54%)



  1%|          | 5/468 [00:00<00:11, 40.37it/s]


Test set: Average loss: 0.0003, Accuracy: 9871/10000 (98.71%)



100%|██████████| 468/468 [00:13<00:00, 35.36it/s]


...


train set: Average loss: 0.0000, Accuracy: 59868/60000 (99.78%)



  1%|          | 4/468 [00:00<00:12, 35.79it/s]


Test set: Average loss: 0.0003, Accuracy: 9871/10000 (98.71%)



100%|██████████| 468/468 [00:13<00:00, 35.09it/s]



train set: Average loss: 0.0000, Accuracy: 59881/60000 (99.80%)


Test set: Average loss: 0.0003, Accuracy: 9873/10000 (98.73%)



#### Q5:
Please print the training and testing accuracy.

In [7]:
print('\nmax testing accuracy: {:.2f}%'.format(
        100. * max_test_correct / len(test_loader.dataset)))
print('\ncorresponding training accuracy: {:.2f}%'.format(
        100. * corresponding_train_correct / len(train_loader.dataset)))


max testing accuracy: 98.78%

corresponding training accuracy: 99.79%
